In [ ]:
%pip install rdflib pandas spacy vaderSentiment requests
!python -m spacy download en_core_web_sm

In [ ]:
# Required Libraries
import pandas as pd
import re
import csv
import time
import requests
import spacy
from spacy.matcher import PhraseMatcher
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD

### Please define the path files here ###


In [ ]:
# Define paths to the data files and outpout files
TEAM_ID = "12"
BASE_PATH = r"Put yout path here"  # <-- Update this to your actual base path

# Data files
NUTRITION_CSV = BASE_PATH + r"\data\Nutrition.csv"
RECIPES_CSV = BASE_PATH + r"\data\Recipes.csv"
RESTAURANTS_CSV = BASE_PATH + r"\data\Restaurants.csv"
REVIEWS_TXT = BASE_PATH + r"\data\Reviews.txt"

# Output files (as per assignment requirements)
STRUCTURED_KG = BASE_PATH + f"\\KEN4256-structured-KG-{TEAM_ID}.ttl"
UNSTRUCTURED_KG = BASE_PATH + f"\\KEN4256-unstructured-KG-{TEAM_ID}.ttl"
INTEGRATED_KG = BASE_PATH + f"\\KEN4256-integrated-KG-{TEAM_ID}.ttl"
SPARQL_RESULTS = BASE_PATH + "\\SPARQL_Results.txt"

In [ ]:
g = Graph()

# Using schema.org as the namespace for our knowledge graph
SDO = Namespace("https://schema.org/")
g.bind("schema", SDO)

# Classes
g.add((SDO.Recipe, RDF.type, OWL.Class))
g.add((SDO.Review, RDF.type, OWL.Class))
g.add((SDO.Nutrition, RDF.type, OWL.Class))
g.add((SDO.Restaurant, RDF.type, OWL.Class))
g.add((SDO.Ingredient, RDF.type, OWL.Class))  
g.add((SDO.Cuisine, RDF.type, OWL.Class))

# Recipe properties
g.add((SDO.image, RDF.type, OWL.DatatypeProperty))
g.add((SDO.image, RDFS.domain, SDO.Recipe))
g.add((SDO.image, RDFS.range, XSD.anyURI))

g.add((SDO.keywords, RDF.type, OWL.DatatypeProperty))
g.add((SDO.keywords, RDFS.domain, SDO.Recipe))
g.add((SDO.keywords, RDFS.range, XSD.string))

g.add((SDO.datePublished, RDF.type, OWL.DatatypeProperty))
g.add((SDO.datePublished, RDFS.domain, SDO.Recipe))
g.add((SDO.datePublished, RDFS.range, XSD.dateTime))

g.add((SDO.cookTime, RDF.type, OWL.DatatypeProperty))
g.add((SDO.cookTime, RDFS.domain, SDO.Recipe))
g.add((SDO.cookTime, RDFS.range, XSD.string))

g.add((SDO.ingredients, RDF.type, OWL.DatatypeProperty))
g.add((SDO.ingredients, RDFS.domain, SDO.Recipe))
g.add((SDO.ingredients, RDFS.range, XSD.string))

g.add((SDO.givenName, RDF.type, OWL.DatatypeProperty))
g.add((SDO.givenName, RDFS.domain, SDO.Recipe))
g.add((SDO.givenName, RDFS.range, XSD.string))

g.add((SDO.recipeCategory, RDF.type, OWL.DatatypeProperty))
g.add((SDO.recipeCategory, RDFS.domain, SDO.Recipe))
g.add((SDO.recipeCategory, RDFS.range, XSD.string))

# Recipe object properties
g.add((SDO.hasNutrition, RDF.type, OWL.ObjectProperty))
g.add((SDO.hasNutrition, RDFS.domain, SDO.Recipe))
g.add((SDO.hasNutrition, RDFS.range, SDO.Nutrition))

g.add((SDO.servesCuisine, RDF.type, OWL.ObjectProperty))
g.add((SDO.servesCuisine, RDFS.domain, SDO.Restaurant))
g.add((SDO.servesCuisine, RDFS.domain, SDO.Recipe))  
g.add((SDO.servesCuisine, RDFS.range, SDO.Cuisine))

# Recipe to Ingredient relationship (Task 3.3)
g.add((SDO.recipeIngredient, RDF.type, OWL.ObjectProperty))
g.add((SDO.recipeIngredient, RDFS.domain, SDO.Recipe))
g.add((SDO.recipeIngredient, RDFS.range, SDO.Ingredient))

g.add((SDO.UserReview, RDF.type, OWL.ObjectProperty))
g.add((SDO.UserReview, RDFS.domain, SDO.Review))
g.add((SDO.UserReview, RDFS.range, SDO.Recipe))

# Nutrition properties
g.add((SDO.calories, RDF.type, OWL.DatatypeProperty))
g.add((SDO.calories, RDFS.domain, SDO.Nutrition))
g.add((SDO.calories, RDFS.range, XSD.decimal))

g.add((SDO.sugarContent, RDF.type, OWL.DatatypeProperty))
g.add((SDO.sugarContent, RDFS.domain, SDO.Nutrition))
g.add((SDO.sugarContent, RDFS.range, XSD.decimal))

g.add((SDO.proteinContent, RDF.type, OWL.DatatypeProperty))
g.add((SDO.proteinContent, RDFS.domain, SDO.Nutrition))
g.add((SDO.proteinContent, RDFS.range, XSD.decimal))

g.add((SDO.fiberContent, RDF.type, OWL.DatatypeProperty))
g.add((SDO.fiberContent, RDFS.domain, SDO.Nutrition))
g.add((SDO.fiberContent, RDFS.range, XSD.decimal))

g.add((SDO.name, RDF.type, OWL.DatatypeProperty))
g.add((SDO.name, RDFS.domain, SDO.Nutrition))
g.add((SDO.name, RDFS.range, XSD.string))

# Restaurant properties
g.add((SDO.AggregateRating, RDF.type, OWL.DatatypeProperty))
g.add((SDO.AggregateRating, RDFS.domain, SDO.Restaurant))
g.add((SDO.AggregateRating, RDFS.range, XSD.decimal))

g.add((SDO.priceRange, RDF.type, OWL.DatatypeProperty))
g.add((SDO.priceRange, RDFS.domain, SDO.Restaurant))
g.add((SDO.priceRange, RDFS.range, XSD.decimal))

g.add((SDO.city, RDF.type, OWL.DatatypeProperty))
g.add((SDO.city, RDFS.domain, SDO.Restaurant))
g.add((SDO.city, RDFS.range, XSD.string))

g.add((SDO.country, RDF.type, OWL.DatatypeProperty))
g.add((SDO.country, RDFS.domain, SDO.Restaurant))
g.add((SDO.country, RDFS.range, XSD.string))

g.add((SDO.orderDelivery, RDF.type, OWL.DatatypeProperty))
g.add((SDO.orderDelivery, RDFS.domain, SDO.Restaurant))
g.add((SDO.orderDelivery, RDFS.range, XSD.integer))

g.add((SDO.legalName, RDF.type, OWL.DatatypeProperty))
g.add((SDO.legalName, RDFS.domain, SDO.Restaurant))
g.add((SDO.legalName, RDFS.range, XSD.string))

# Review properties
g.add((SDO.reviewRating, RDF.type, OWL.DatatypeProperty))
g.add((SDO.reviewRating, RDFS.domain, SDO.Review))
g.add((SDO.reviewRating, RDFS.range, XSD.string))


g.add((SDO.reviewCount, RDF.type, OWL.DatatypeProperty))
g.add((SDO.reviewCount, RDFS.domain, SDO.Review))
g.add((SDO.reviewCount, RDFS.range, XSD.integer))



# Using givenName , name and legalName as properties for the name of the recipe, nutrition and restaurant respectively
# so we unbind the default name property from schema.org to avoid confusion and ensure that we use the correct properties for each class

# Serializing the knowledge graph to a turtle file so we can visualize it with https://service.tib.eu/webvowl/ 
g.serialize("food_nutrition_kg.ttl", format="turtle")

In [ ]:
# Using custom namespaces for our knowledge graph, because if we just use the fod namespace for everything  the recipe and nutrition 
# URIs colide in the namespace and the instances of recipes and nutrition become one. By using separate namespaces 
# we can better structure our knowledge graph and make it easier to understand and query
FOD = Namespace("http://kg-course.io/food-nutrition/")
g.bind("fod", FOD)  
NUTR = Namespace("http://kg-course.io/food-nutrition/nutrition/")
g.bind("nutr", NUTR)
REC = Namespace("http://kg-course.io/food-nutrition/recipe/")
g.bind("rec", REC)

In [ ]:
# Function to create safe URIs by removing/encoding problematic characters, because we hade issues with special characters 
# in the data causing problems when creating URIs for the instances
def clean_for_uri(name):
    """Clean a string to make it safe for use in a URI."""
    if pd.isna(name):
        return "Unknown"
    name = str(name).strip()
    # Remove quotes and parentheses
    name = name.replace('"', '').replace("'", "").replace("(", "").replace(")", "")
    # Replace spaces and commas with underscores
    name = name.replace(" ", "_").replace(",", "_")
    # Remove or replace other problematic characters
    name = re.sub(r'[{}%#?&/\\<>|:*]', '', name)
    # Remove any non-ASCII characters that could cause issues
    name = name.encode('ascii', 'ignore').decode('ascii')
    # Remove multiple underscores
    name = re.sub(r'_+', '_', name)
    # Remove leading/trailing underscores
    name = name.strip('_')
    return name if name else "Unknown"

#Creating all data frames with pandas so we can easily go through them and add data to the knowledge graph
nutrion_df = pd.read_csv(
    NUTRITION_CSV,
    sep=";",
    encoding="latin1"
)

print(nutrion_df.head())

# Had trouble with data from recipes file, because of multi-line cells. So we use the Python engine and specify the quote character
recipes_df = pd.read_csv(
    RECIPES_CSV,
    sep=";",
    encoding="latin1",
    #Trouble with multi-line cells, so we use the Python engine and specify the quote character to handle the c("") blocks properly
    engine='python',        
    quotechar='"',          
    # default is doublequote=True, but we specify it explicitly to ensure that the "" inside the c("") blocks are handled correctly
    doublequote=True,       
    # In case of a bad line we skip it
    on_bad_lines='skip'     
)

print(recipes_df.head())


restaurants_df = pd.read_csv(
    RESTAURANTS_CSV,
    sep=";",
    encoding="latin1", 
)

#restaurants_df.head()

In [ ]:
# Itterate over the nutrition data to populate the knowledge graph with them
for _,row in nutrion_df.iterrows():

    # Transform the names into URIs
    clean_name = clean_for_uri(row["Name"])
    # Using "http://kg-course.io/food-nutrition/nutrition" as the namespace for the food items 
    # We create a URI for each food item and add it to the graph as an instance of the Nutrition class
    nutrition_uri = NUTR[clean_name]
    # Add the nutrition information to the graph as RDF triples, name, calories, sugar content, protein content, and fiber content are added
    # as properties from schema.org
    g.add((nutrition_uri, RDF.type, SDO.Nutrition))
    g.add((nutrition_uri, SDO.name, Literal(row["Name"], datatype=XSD.string)))
    g.add((nutrition_uri, SDO.calories, Literal(row["Calories"], datatype=XSD.decimal)))
    g.add((nutrition_uri, SDO.sugarContent, Literal(row["SugarContent"], datatype=XSD.decimal)))
    g.add((nutrition_uri, SDO.proteinContent, Literal(row["ProteinContent"], datatype=XSD.decimal)))
    g.add((nutrition_uri, SDO.fiberContent, Literal(row["FiberContent"], datatype=XSD.decimal)))

    # We only take the first 10000 rows to manage computational complexity and avoid scalability issuses
    if row.name > 10000:
        break

print(g.serialize(format='ttl'))

In [ ]:
#Extracting the first image from the urls so we can output it in the sparql querry at the end
def extract_first_image_url(images_str):
    if pd.isna(images_str) or str(images_str).strip() == "" or "character(0)" in str(images_str):
        return ""
    images_str = str(images_str)
    # Find URLs inside double quotes to preserve commas in URL 
    urls = re.findall(r'"(https?://[^"]+)"', images_str)
    if urls:
        return urls[0]
    return ""

print("This part is computationally intensive and may take a few minutes")
for _,row in recipes_df.iterrows():

    # Transform the names into URIs as above
    clean_name = clean_for_uri(row["Name"])
    recipe_uri = REC[clean_name]
    
    # Bind the recipe data to the knowledge graph by adding RDF triples for the recipe properties, such as recipe name(givenName), image, keywords, date published, cook time, and ingredients
    g.add((recipe_uri, RDF.type, SDO.Recipe))
    g.add((recipe_uri, SDO.givenName, Literal(row["Name"], datatype=XSD.string)))
    g.add((recipe_uri, SDO.image, Literal(extract_first_image_url(row["Images"]), datatype=XSD.anyURI)))
    g.add((recipe_uri, SDO.keywords, Literal(row["Keywords"], datatype=XSD.string)))
    g.add((recipe_uri, SDO.datePublished, Literal(row["DatePublished"], datatype=XSD.dateTime)))
    g.add((recipe_uri, SDO.cookTime, Literal(row["CookTime"], datatype=XSD.string)))
    g.add((recipe_uri, SDO.ingredients, Literal(row["RecipeIngredientParts"], datatype=XSD.string)))
    g.add((recipe_uri, SDO.recipeCategory, Literal(row["RecipeCategory"], datatype=XSD.string)))


    # Link the recipe to nutrition information based on the name of the recipe and the name of the nutrition 
    # If the name of the recipe contains the name of a nutrition item, we link them with the hasNutrition property. 
    if row["Name"] in nutrion_df["Name"].values:
        clean_nutr_name = clean_for_uri(row["Name"])
        nutrition_uri = NUTR[clean_nutr_name]
        g.add((recipe_uri, SDO.hasNutrition, nutrition_uri))

    # Link the recipe to cuisines based on the keywords and the cuisines in the restaurants dataset
    keywords_raw = row["Keywords"]
    # Ensure Keywords is a string and not NaN because the data has some missing values in the Keywords column
    if pd.notna(keywords_raw):
        for keyword in str(keywords_raw).split(","):
            # Trouble with keywords containing c(""), so we clean them first
            # Clean keywords remove quotes, c(), parentheses so it can match to cuisines
            keyword = keyword.strip().replace('"', '').replace("'", "").replace("c(", "").replace(")", "").strip()
            
            # Skip empty keywords
            if not keyword:
                continue
                
            # Check if this keyword appears in any restaurant cuisine ,partial match
            matching_cuisines = restaurants_df[restaurants_df['Cuisines'].str.contains(keyword, case=False, na=False, regex=False)]
            if len(matching_cuisines) > 0:
                # Use a consistent naming convention for the URI
                clean_kw = clean_for_uri(keyword)
                keywords_uri = FOD[clean_kw]
                g.add((recipe_uri, SDO.servesCuisine, keywords_uri))
    
    if row.name > 10000:
        break

print(g.serialize(format="turtle"))

In [ ]:
# Itterate over the restaurant data as we did with recipes and nutrition.
for _, row in restaurants_df.iterrows():

    clean_name = clean_for_uri(row["Restaurant Name"])
    restaurant_uri = FOD[clean_name]
    
    # Adding restaurant instances to the graph with their properties 
    # such as legal name, aggregate rating, price range, city, country, online delivery and cuisines served
    g.add((restaurant_uri, RDF.type, SDO.Restaurant))
    g.add((restaurant_uri, SDO.legalName, Literal(row["Restaurant Name"], datatype=XSD.string)))
    g.add((restaurant_uri, SDO.AggregateRating, Literal(row["Aggregate rating"], datatype=XSD.decimal)))
    g.add((restaurant_uri, SDO.priceRange, Literal(row["Price range"], datatype=XSD.decimal)))
    g.add((restaurant_uri, SDO.city, Literal(row["City"], datatype=XSD.string)))
    g.add((restaurant_uri, SDO.country, Literal(row["Country"], datatype=XSD.string)))
    g.add((restaurant_uri, SDO.orderDelivery, Literal(row["Has Online delivery"], datatype=XSD.integer)))
    g.add((restaurant_uri, SDO.servesCuisine, Literal(row["Cuisines"], datatype=XSD.string)))
    
    if row.name > 10000:
        break

print(g.serialize(format='ttl'))

In [ ]:
# Save the complete structured KG with all entities (recipes, nutrition, restaurants)
g.serialize(destination=STRUCTURED_KG, format="turtle")

In [ ]:
MAX_REVIEWS = 1000
NUM_RECIPES = 1000

nlp = spacy.load("en_core_web_sm")
analyzer = SentimentIntensityAnalyzer()

# 1. Build RecipeId -> Recipe Name lookup from first NUM_RECIPES recipes
recipe_subset = recipes_df.head(NUM_RECIPES)

# clean recipe name for URI
recipe_id_to_name = {}
for _, row in recipe_subset.iterrows():
    recipe_id = str(row["RecipeId"])
    recipe_name = clean_for_uri(row["Name"])
    recipe_id_to_name[recipe_id] = recipe_name


# 2. Vocabulary building for ingredient extraction
vocab_set = set()
for _, row in recipe_subset.iterrows():
    raw_parts = str(row.get("RecipeIngredientParts", ""))
    parts = re.findall(r'"([^"]*)"', raw_parts) if 'c(' in raw_parts else [raw_parts]
    
    for p in parts:
        doc = nlp(p.lower())
        lemmas = [t.lemma_ for t in doc if t.pos_ in ["NOUN", "PROPN"] and not t.is_stop]
        if lemmas: 
            vocab_set.add(" ".join(lemmas))

matcher = PhraseMatcher(nlp.vocab, attr="LEMMA")
matcher.add("INGREDIENT", list(nlp.pipe(vocab_set)))

# 3. Review Processing and matching via RecipeId
g_unstructured = Graph()
g_unstructured.bind("schema", SDO)
g_unstructured.bind("rec", REC)
g_unstructured.bind("fod", FOD)

# Procesing the reviews file line by line
matched_reviews = 0
with open(REVIEWS_TXT, "r", encoding="utf-8", errors="replace") as f:
    reader = csv.reader(f, delimiter="\t", quotechar='"')
    count = 0
    for row in reader:
        if count >= MAX_REVIEWS: break
        if len(row) < 5: continue
        
        # Columns: ReviewId, RecipeId, AuthorId, AuthorName, Review, ...
        raw_review_id = row[0]
        raw_recipe_id = row[1]  # Use RecipeId (not AuthorId!)
        review_text = row[4]

        # Skip header row
        if raw_review_id.lower() == "reviewid": continue

        # Look up recipe name from RecipeId
        if raw_recipe_id not in recipe_id_to_name:
            # Recipe not in our subset, skip this review
            continue
        
        recipe_name = recipe_id_to_name[raw_recipe_id]
        matched_reviews += 1

        safe_rid = clean_for_uri(raw_review_id)
        review_uri = FOD[f"review_{safe_rid}"]
        recipe_uri = REC[recipe_name]
        
        g_unstructured.add((review_uri, RDF.type, SDO.Review))
        g_unstructured.add((review_uri, SDO.UserReview, recipe_uri))
        
        # Sentiment Analysis
        score = analyzer.polarity_scores(review_text)["compound"]
        sentiment = "positive" if score > 0.05 else "negative" if score < -0.05 else "neutral"
        g_unstructured.add((review_uri, SDO.reviewRating, Literal(sentiment, datatype=XSD.string)))

        # Mention extraction
        doc = nlp(review_text)
        for _, start, end in matcher(doc):
            term = doc[start:end].lemma_
            g_unstructured.add((review_uri, SDO.keywords, Literal(term, datatype=XSD.string)))

        count += 1

# 4. Final Serialization
try:
    g_unstructured.serialize(destination=UNSTRUCTURED_KG, format="turtle")
    print(f"File created successfully: {UNSTRUCTURED_KG}")
except Exception as e:
    print(f"Serialization Error: {e}")

In [ ]:
WD = Namespace("http://www.wikidata.org/entity/")
WIKIDATA_KG = BASE_PATH + f"\\KEN4256-wikidata-KG-{TEAM_ID}.ttl"

g_wikidata = Graph()
g_wikidata.bind("wd", WD)
g_wikidata.bind("owl", OWL)
g_wikidata.bind("schema", SDO)
g_wikidata.bind("fod", FOD)
g_wikidata.bind("rec", REC)

# Link Ingredients to Wikidata
def search_wikidata(term):
    """Search Wikidata for a term and return QID."""
    try:
        r = requests.get("https://www.wikidata.org/w/api.php",
            params={"action": "wbsearchentities", "search": term, "language": "en", "format": "json", "limit": 1},
            headers={"User-Agent": "KG-Course-Assignment/1.0"},  # <-- THIS LINE WAS MISSING
            timeout=10)
        data = r.json()
        if data.get("search"):
            return data["search"][0]["id"]
        return None
    except:
        return None
# Extract unique ingredient names from the recipes dataset to link them to Wikidata. 
# We clean the ingredient names and filter out unreasonable ones to improve matching accuracy.
ingredient_words = set()
for _, row in recipes_df.head(NUM_RECIPES).iterrows():
    raw_parts = str(row.get("RecipeIngredientParts", ""))
    # Extract individual ingredient names from c("...", "...", ...) format
    parts = re.findall(r'"([^"]*)"', raw_parts) if 'c(' in raw_parts else [raw_parts]
    for p in parts:
        # Clean and add if it's a reasonable ingredient name
        p = p.strip().lower()
        # Allow letters, spaces, and hyphens (e.g. "vanilla yogurt", "lemon juice")
        if len(p) >= 3 and len(p) <= 30 and p.replace(" ", "").replace("-", "").isalpha():
            ingredient_words.add(p)


# Link to Wikidata
ingredient_links = 0
# We limit to 50 ingredients for demonstration purposes, but in a real scenario we would want to link as many as possible while respecting rate limits and computational resources.
for ingredient in list(ingredient_words)[:50]:  # Limit to 50
    qid = search_wikidata(ingredient)
    if qid:
        ing_uri = FOD[f"ingredient_{clean_for_uri(ingredient)}"]
        g_wikidata.add((ing_uri, RDF.type, SDO.Ingredient))
        g_wikidata.add((ing_uri, RDFS.label, Literal(ingredient, lang="en")))
        g_wikidata.add((ing_uri, OWL.sameAs, WD[qid]))
        ingredient_links += 1
        print(f"  Linked: {ingredient} -> wd:{qid}")
    time.sleep(0.4)  # Rate limiting

print(f"\nLinked {ingredient_links} ingredients to Wikidata")

# Query Wikidata for Recipe-Cuisine Relationships
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

# Get cuisines and their countries of origin
cuisine_query = """
SELECT ?cuisine ?cuisineLabel ?country ?countryLabel WHERE {
  ?cuisine wdt:P31 wd:Q1778821 .
  OPTIONAL { ?cuisine wdt:P495 ?country . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} LIMIT 100
"""

try:
    r = requests.get(WIKIDATA_ENDPOINT, 
        params={"query": cuisine_query}, 
        headers={"Accept": "application/sparql-results+json", "User-Agent": "KG-Course-Assignment/1.0"},
        timeout=60)
    
    if r.status_code == 200:
        results = r.json()["results"]["bindings"]
        for item in results:
            qid = item["cuisine"]["value"].split("/")[-1]
            label = item.get("cuisineLabel", {}).get("value", "Unknown")
            
            cuisine_uri = FOD[f"cuisine_{clean_for_uri(label)}"]
            g_wikidata.add((cuisine_uri, RDF.type, SDO.Cuisine))
            g_wikidata.add((cuisine_uri, RDFS.label, Literal(label, lang="en")))
            g_wikidata.add((cuisine_uri, OWL.sameAs, WD[qid]))
            
            if "countryLabel" in item:
                country = item["countryLabel"]["value"]
                g_wikidata.add((cuisine_uri, SDO.country, Literal(country, datatype=XSD.string)))
        
except Exception as e:
    print(f"  Error fetching cuisines: {e}")

time.sleep(2)

# Get dishes and their cuisines
print("\nFetching dish-cuisine relationships...")
dish_query = """
SELECT ?dish ?dishLabel ?cuisine ?cuisineLabel WHERE {
  ?dish wdt:P31 wd:Q746549 .
  ?dish wdt:P361 ?cuisine .
  ?cuisine wdt:P31 wd:Q1778821 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} LIMIT 200
"""

# We query Wikidata for dishes (instances of Q746549) and their associated cuisines (which are instances of Q1778821).
try:
    r = requests.get(WIKIDATA_ENDPOINT, 
        params={"query": dish_query}, 
        headers={"Accept": "application/sparql-results+json", "User-Agent": "KG-Course-Assignment/1.0"},
        timeout=60)
    
    # If the request is successful, we process the results and add them to our Wikidata KG. 
    # We create URIs for dishes and cuisines, link them with the servesCuisine property, and also link to their Wikidata QIDs using owl:sameAs.
    if r.status_code == 200:
        results = r.json()["results"]["bindings"]
        for item in results:
            dish_qid = item["dish"]["value"].split("/")[-1]
            dish_label = item.get("dishLabel", {}).get("value", "Unknown")
            cuisine_label = item.get("cuisineLabel", {}).get("value", "Unknown")
            
            dish_uri = FOD[f"dish_{clean_for_uri(dish_label)}"]
            cuisine_uri = FOD[f"cuisine_{clean_for_uri(cuisine_label)}"]
            
            g_wikidata.add((dish_uri, RDF.type, SDO.Recipe))
            g_wikidata.add((dish_uri, RDFS.label, Literal(dish_label, lang="en")))
            g_wikidata.add((dish_uri, OWL.sameAs, WD[dish_qid]))
            g_wikidata.add((dish_uri, SDO.servesCuisine, cuisine_uri))
        
except Exception as e:
    print(f"  Error fetching dishes: {e}")



In [ ]:
# Merge structured, unstructured, and Wikidata KGs into integrated KG
integrated_g = Graph()
integrated_g.parse(STRUCTURED_KG, format="turtle")

# We attempt to load the unstructured and Wikidata KGs, but we handle potential errors gracefully. 
# If there are issues with loading these KGs, we catch the exceptions and print a note instead of crashing the program.
try:
    integrated_g.parse(UNSTRUCTURED_KG, format="turtle")
    print(f"Loaded unstructured KG. Total triples: {len(integrated_g)}")
except Exception as e:
    print(f"Note: Could not load unstructured KG: {e}")

try:
    integrated_g.parse(WIKIDATA_KG, format="turtle")
    print(f"Loaded Wikidata KG. Total triples: {len(integrated_g)}")
except Exception as e:
    print(f"Note: Could not load Wikidata KG: {e}")

integrated_g.serialize(destination=INTEGRATED_KG, format="turtle")


In [ ]:
print(f"LOADING THE GRAPH (This may take 10-20 seconds) ...")
g = rdflib.Graph()
try:
    g.parse(INTEGRATED_KG, format="turtle")
    print(f"Graph ready! Total triples: {len(g)}")
except Exception as e:
    print(f"Error reading file: {e}")
    exit()

# PREFIXES
PREFIXES = """
    PREFIX fod: <http://kg-course.io/food-nutrition/>
    PREFIX rec: <http://kg-course.io/food-nutrition/recipe/>
    PREFIX nutr: <http://kg-course.io/food-nutrition/nutrition/>
    PREFIX schema: <https://schema.org/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
"""

# Open the text file to write results
with open(SPARQL_RESULTS, "w", encoding="utf-8") as f:
    
    def run_query(task_id, description, query_text):
        print(f"\n>>> Processing Task {task_id}: {description}...")
        
        # Write Header to file
        f.write(f"\n{'='*50}\n")
        f.write(f"Task {task_id}: {description}\n")
        f.write(f"{'='*50}\n")
        
        try:
            t0 = time.time()
            results = g.query(PREFIXES + query_text)
            dt = time.time() - t0
            
            data = []
            for row in results:
                clean_row = []
                for item in row:
                    if item is None:
                        clean_row.append("N/A")
                    else: 
                        val = str(item)
                        # Keep only actual image URLs intact (not KG URIs)
                        if ('http://' in val or 'https://' in val) and ('.jpg' in val or '.png' in val):
                            clean_row.append(val)
                        elif val.strip() == '':
                            clean_row.append("")
                        else:
                            clean_row.append(val.split('/')[-1].replace("_", " "))
                data.append(clean_row)
                
            cols = [str(v) for v in results.vars]
            df = pd.DataFrame(data, columns=cols)
            
            if df.empty:
                msg = "(No results found in this sample)"
                print(f"   {msg}")
                f.write(f"{msg}\n")
            else:
                # Convert DataFrame to string table
                table_str = df.head(20).to_string(index=False)
                print(df.head(5).to_string(index=False)) # Show preview in terminal
                f.write(table_str + "\n") # Write full table to file
                print(f"   (Done in {dt:.2f}s)")
                
        except Exception as e:
            err_msg = f"Error: {e}"
            print(f"   {err_msg}")
            f.write(f"{err_msg}\n")

    # QUERIES

    # 4.1
    run_query("4.1", "Recipes with Mango", """
    SELECT ?Name WHERE {
        ?recipe a schema:Recipe ; schema:givenName ?Name ; schema:ingredients ?ing .
        FILTER(CONTAINS(LCASE(?ing), "mango"))
    } LIMIT 30
    """)

    # 4.2
    run_query("4.2", "Quick and Healthy Pies", """
    SELECT ?Name ?Time WHERE {
        ?recipe a schema:Recipe ; schema:givenName ?Name ; schema:keywords ?k ; schema:cookTime ?Time .
        FILTER(CONTAINS(LCASE(?k), "healthy") && CONTAINS(LCASE(?Name), "pie"))
        FILTER(!CONTAINS(?Time, "H"))
        FILTER(!CONTAINS(LCASE(STR(?Time)), "nan"))
    } LIMIT 30
    """)

    # 4.3
    run_query("4.3", "Chinese Restaurants New Delhi", """
    SELECT ?Restaurant WHERE {
        ?rest a schema:Restaurant ; schema:legalName ?Restaurant ; 
            schema:city ?city ; schema:orderDelivery ?delivery ; schema:servesCuisine ?c .
        FILTER(STR(?city) = "New Delhi")
        FILTER(?delivery = 1)
        FILTER(CONTAINS(LCASE(STR(?c)), "chinese"))
    } LIMIT 10
    """)

    # 4.4
    run_query("4.4", "Average Cost (Davenport Asian)", """
    SELECT (AVG(?price) AS ?Average_Cost) WHERE {
        ?rest a schema:Restaurant ; schema:city ?city ; 
            schema:priceRange ?price ; schema:servesCuisine ?c .
        FILTER(STR(?city) = "Davenport")
        FILTER(REGEX(STR(?c), "Asian|Chinese|Thai", "i"))
    }
    """)

    # 4.5
    run_query("4.5", "Easy Desserts (<300cal, after 2000)", """
    SELECT ?Name ?Calories ?Image WHERE {
        ?recipe a schema:Recipe ; schema:givenName ?Name ; schema:recipeCategory ?cat ; 
                schema:keywords ?k ; schema:hasNutrition ?n ; schema:image ?Image ; schema:datePublished ?date .
        ?n schema:calories ?Calories .
        FILTER(REGEX(STR(?cat), "dessert", "i") && REGEX(STR(?k), "easy", "i") && ?Calories < 300)
        FILTER(CONTAINS(STR(?date), "200") || CONTAINS(STR(?date), "201") || CONTAINS(STR(?date), "202"))
    } ORDER BY ASC(?Calories) LIMIT 5
    """)

    # 4.6
    run_query("4.6", "Top Beverages (Sentiment)", """
    SELECT ?Name ?PrepTime ?Sugar (SAMPLE(?Sentiment) AS ?SentimentSample) WHERE {
        ?recipe a schema:Recipe ; schema:givenName ?Name ; schema:recipeCategory ?cat ; 
                schema:cookTime ?PrepTime ; schema:hasNutrition ?n .
        ?n schema:sugarContent ?Sugar .
        ?review a schema:Review ; schema:UserReview ?recipe ; schema:reviewRating ?Sentiment .
        FILTER(CONTAINS(LCASE(?cat), "beverage"))
    } GROUP BY ?Name ?PrepTime ?Sugar
    ORDER BY DESC(?Sugar) LIMIT 10
    """)

    # 4.7
    run_query("4.7", "High Protein Recipes", """
    SELECT ?Name ?Protein ?Cuisine WHERE {
        ?recipe a schema:Recipe ; schema:givenName ?Name ; schema:hasNutrition ?nut .
        ?nut schema:proteinContent ?Protein .
        FILTER(?Protein > 20)
        OPTIONAL {
            ?recipe schema:servesCuisine ?cuisineUri .
            BIND(STR(?cuisineUri) AS ?Cuisine)
        }
    } ORDER BY DESC(?Protein) LIMIT 10
    """)

    # 4.8
    run_query("4.8", "Top Healthy (NDS Score)", """
    SELECT ?Name ?NDS ?Protein ?Fiber ?Sugar WHERE {
        ?recipe a schema:Recipe ; schema:givenName ?Name ; schema:hasNutrition ?n .
        ?n schema:proteinContent ?p ; schema:fiberContent ?f ; schema:sugarContent ?s .
        BIND(?p AS ?Protein)
        BIND(?f AS ?Fiber)
        BIND(?s AS ?Sugar)
        BIND ((1.0 * ?p + 1.5 * ?f - 2.0 * ?s) AS ?NDS)
    } ORDER BY DESC(?NDS) LIMIT 5
    """)

print(f"\n SUCCESS! Results saved to '{SPARQL_RESULTS}' ")